# Solution for Pandas Task: Exploratory Analysis of Economic and Social Data

In this task, we explore economic and social data using pandas.
Therefore, we combine two different data sources: a CSV file and a SQLite database.


In [ ]:
import pandas as pd
import sqlite3


In [ ]:
SOCIAL_CSV_PATH = "./data/social_indicators.csv"
ECONOMIC_DB_PATH = "./data/economic_indicators.db"


## Data Loading

**Task:** Use pandas to load the CSV file with social indicators into a dataframe.
`SOCIAL_CSV_PATH` provides the file path.
Please note that the first column contains index values.


In [ ]:
df_social = pd.read_csv(SOCIAL_CSV_PATH, index_col=0)
df_social.head(5)


**Task:** Use the database connector `db_con` to load the economic data from the database. More precisely, write a query to load the whole table `economic_statistics`.


In [ ]:
db_con = sqlite3.connect(ECONOMIC_DB_PATH)
df_economic = pd.read_sql(sql="SELECT * FROM economic_statistics;", con=db_con)
df_economic.head(5)


## Data Preprocessing

**Task:** Merge both data frames using the `country_name` column.


In [ ]:
df_indicators = df_social.merge(df_economic, on="country_name", how="inner")
df_indicators.head(5)


Let's have a look at the meta data.


In [ ]:
df_indicators.info()


There are some unknown values, hindering to infer integer columns.

**Task:** Drop unknown values


In [ ]:
df_indicators.dropna(inplace=True)


Let's use more specific data types.


In [ ]:
df_indicators = df_indicators.astype(
    {'country_name': 'string', 'population_total': 'int64', 'population_urban': 'int64', 'labor_force_total': 'int64'})
df_indicators.info()


**Task:** Describe the numerical columns with general statistics.


In [ ]:
df_indicators.describe()


## Exploratory Data Analysis

Please answer the following questions using our sample `df_indicators`.

**Question 1:** How many people live in a city?


In [ ]:
df_indicators.population_urban.sum()


**Question 2:** How many people are unemployed on average? Add a new column `unemployed_total` and calculate its mean.


In [ ]:
df_indicators['unemployed_total'] = df_indicators.population_total - \
    df_indicators.labor_force_total
df_indicators.unemployed_total.mean()


**Question 3:** How many countries have higher export rates than import rates? Hint: You can sum up `bool` values, where `True=1` and `False=0`.


In [ ]:
df_indicators['more_export_than_import'] = df_indicators.exports_perc_of_gdp > df_indicators.imports_perc_of_gdp
df_indicators.more_export_than_import.sum()


Let's define the `development_index` column as followed: $\frac{e}{p} + l$, where $e$ is the total expenditure on education (use the GDP to calculate it), $p$ is the total population, and $l$ is the life expectancy at birth.


In [ ]:
df_indicators['development_index'] = df_indicators.government_expenditure_on_education_perc_of_gdp * \
    df_indicators.gdp_us_dollar / df_indicators.population_total + \
    df_indicators.life_expectancy_at_birth


**Question 4:** Which ten countries have the highest `development_index`? Only display `country_name` and `development_index`.


In [ ]:
df_indicators.sort_values(by='development_index', ascending=False)[
    ['country_name', 'development_index']].head(10)


**Bonus task:** We define the `development_level` as a category for the top, medium and low third of all countries regarding the `development_index`. How many people live in countries with a respective `development_level`?


In [ ]:
df_indicators['development_level'] = pd.cut(
    x=df_indicators.development_index, bins=3, labels=["low", "medium", "top"])
df_indicators[['country_name', 'development_index',
               'development_level']].sort_values(by='development_index')


In [ ]:
df_indicators.groupby(by='development_level').agg({'population_total': 'sum'})
